In [1]:
from market_analysis.models import Candle, UserProfile, Symbol, SortedStocksList, TickerData, StrategyTimestamp
# from market_analysis import views
from django.contrib.auth.models import User
from heystox_intraday.select_stocks_for_trading import *
import pandas as pd
# from heystox_trade.settings import *
from datetime import datetime, timedelta

In [3]:
# obj = Candle.objects.filter(symbol__symbol="ambujacem", candle_type="M5", date__range=[datetime.strptime("17/10/2019",'%d/%m/%Y').date(), datetime.strptime("19/10/2019",'%d/%m/%Y').date()]).values()
# # # new_header = ["Exchange", "Token", "NA", "Symbol", "Full Name", "closed_price", "NA1", "NA2", "Tick Size", "Lot Size", "Instrument Type", "ISIN"]

# df = pd.DataFrame(list(obj))
# # # obj.columns = new_header
# # # new_obj = obj[(obj.close_price >= 55) & (obj.close_price <= 250)]
# symbol = Symbol.objects.get(symbol="pnb")
# # symbol.is_stock_ohl(date="17/10/2019")
# first_candle_price = df.iloc[0:]
# # previouse_day_last_candle_price = df.loc[:, ["open_price", "high_price", "low_price", "close_price"]]
# # min(current_candle_price.min().values)
# last_day_candles = df[(df.date <= datetime.strptime("18/10/2019",'%d/%m/%Y').date())]
# last_day_closing_price = last_day_candles.iloc[[-1]].close_price
# last_day_opening_price = last_day_candles.iloc[[0]].open_price
# today_open_price = df[(df.date >= datetime.strptime("18/10/2019",'%d/%m/%Y').date())].iloc[[0]].open_price
# # print(first_candle_price)
# today_open_price > 100


In [4]:
import numpy as np

In [5]:
from ta.trend import macd, macd_diff, macd_signal, ema, ema_indicator
from ta.momentum import stoch, stoch_signal
# stock = Symbol.objects.get(symbol="ibulhsgfin")
# obj = stock.get_stock_data(days=1, end_date=datetime.strptime("30/10/2019",'%d/%m/%Y').date())
# df = pd.DataFrame(list(obj.values()))
# df["macd"] = macd(df.close_price)
# df["macd_signal"] = macd_signal(df.close_price)
# df["diff"] = macd_diff(df.close_price)
# df.to_csv("stoch_analytics.csv")
# df.head(2)
# obj.values()
from django.core.cache import caches, cache
from heystox_intraday.intraday_fetchdata import get_upstox_user
from market_analysis.tasks.day_trading_tasks import *
from heystox_intraday.select_stocks_for_trading import *

In [17]:
upstox_user = get_upstox_user("sonupal129@gmail.com")
StrategyTimestamp.objects.filter(timestamp__date=datetime.now().date())

<QuerySet [<StrategyTimestamp: aartidrugs | 2019-12-02 | 23:23:26>]>

In [7]:
# users_tasks done
# Stock data import task function to be tested update_nifty_50_data
# misc.py done
# 



# intraday_fetchdata
# intraday_function_strategy Done
# intraday_strategies Done
# select_stocks_for_trading Done